In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/kaartest1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('../input/kaartest1/kaar train.csv')

In [ ]:
df.columns

In [ ]:
df_orig = df.copy()

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold,StratifiedKFold
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import missingno as msno #Visualize null

sns.set_style('ticks') #No grid with ticks
print(sns.__version__)

In [ ]:
df.info()

In [ ]:
df.isna().any()

In [ ]:
df.info()

In [ ]:
cols=['Gender','Age','Purchase Amount','Sales Amount']
for i in cols:
  print(df[i].value_counts())

In [ ]:
new_data = df.dropna()
new_data.info()

In [ ]:
labelencoder = LabelEncoder()

df_max_scaled = new_data.copy()

## FEATURE ENGINEERING
df_max_scaled = df_max_scaled.astype({
    'Gender' : 'category'
})

cat_cols = [i for i  in df_max_scaled.columns if df_max_scaled[i].dtype not in ['int64', 'float64']]

for col in cat_cols:
  df_max_scaled[col + "-cat"] = labelencoder.fit_transform(df_max_scaled[col])

num_cols = [col for col in df_max_scaled.columns if df_max_scaled[col].dtype in ['int', 'float']]

for i in cat_cols:
  df_max_scaled.drop([i], axis= 1, inplace= True)

for col in num_cols:
  df_max_scaled[col] = df_max_scaled[col] / df_max_scaled[col].abs().max() 

df_max_scaled.head()

In [ ]:
num_cols = [col for col in df_max_scaled.columns if df_max_scaled[col].dtype in ['int64','float64']]

cat_cols = [col for col in df_max_scaled.columns if df_max_scaled[col].dtype not in ['int64', 'float64']]

In [ ]:
def dist_box(data):
 # function plots a combined graph for univariate analysis of continous variable 
 #to check spread, central tendency , dispersion and outliers  
    Name=data.name.upper()
    fig,(ax_box,ax_dis)  =plt.subplots(2,1,gridspec_kw = {"height_ratios": (.25, .75)},figsize=(8, 5))
    mean=data.mean()
    median=data.median()
    mode=data.mode().tolist()[0]
    fig.suptitle("SPREAD OF DATA FOR "+ Name  , fontsize=18, fontweight='bold')
    sns.boxplot(x=data,showmeans=True, orient='h',color="violet",ax=ax_box)
    ax_box.set(xlabel='')
    sns.distplot(data,kde=False,color='blue',ax=ax_dis)
    ax_dis.axvline(mean, color='r', linestyle='--',linewidth=2)
    ax_dis.axvline(median, color='g', linestyle='-',linewidth=2)
    ax_dis.axvline(mode, color='y', linestyle='-',linewidth=2)
    plt.legend({'Mean':mean,'Median':median,'Mode':mode})

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

for i in range(len(num_cols)):
    dist_box(df_max_scaled[num_cols[i]])

In [ ]:
data = df_max_scaled.drop_duplicates()
data.head()

In [ ]:
y = data["Sales Amount"]
X = data.drop('Sales Amount',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [ ]:
df_max_scaled.shape

In [ ]:
from sklearn import preprocessing
from sklearn import utils
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y_train)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [ ]:
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded1 = lab_enc.fit_transform(y_test)
print(utils.multiclass.type_of_target(y_test))
print(utils.multiclass.type_of_target(y_test.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded1))

In [ ]:
m1 = 'Random Forest Classfier'
rf = RandomForestClassifier(n_estimators=20, max_depth=5)
rf.fit(X_train,training_scores_encoded)
rf_predicted = rf.predict(X_test)
rf_conf_matrix = confusion_matrix(training_scores_encoded1, rf_predicted)
rf_acc_score = accuracy_score(training_scores_encoded1, rf_predicted)
print("confussion matrix")
print(rf_conf_matrix)
print("\n")
print("Accuracy of Random Forest:",rf_acc_score*100,'\n')
print(classification_report(training_scores_encoded1,rf_predicted))

kfold = KFold(n_splits=10, random_state=None)
cv_results = cross_val_score(rf, X_train, training_scores_encoded, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (m1, cv_results.mean(), cv_results.std())
print(msg)

In [ ]:
import pickle
pickle.dump(rf, open('rfmodel.pkl','wb'))

In [ ]:
loaded_model=pickle.load(open('./rfmodel.pkl','rb'))

In [ ]:
loaded_model.predict([np.array([1,19,19])])[0]